In [1]:
import urllib2, base64
import pandas as pd

In [2]:
f = open('cred.txt','r')
username = f.readline()
password = f.readline()
f.close()
username = username.strip()
password = password.strip()

#This connects to the webpage and read the html text
def readUrl(url):
    request = urllib2.Request(url)
    base64string = base64.encodestring('%s:%s' % (username, password)).replace('\n', '')
    request.add_header("Authorization", "Basic %s" % base64string)   
    result = urllib2.urlopen(request)
    htmltext = result.read()
    return htmltext

def convertFirstRowToHeader(df):
    df.columns = df.iloc[0]
    df = df[1:]
    return df

In [3]:
f = open('ShibberuCourses.txt','r')
line = f.readline()

In [4]:
df = pd.DataFrame()
# for k in range(0,3):
while line:
    number = line.split(',')
    url = 'https://prodweb.rose-hulman.edu/regweb-cgi/reg-sched.pl?type=Roster&termcode=' + number[0] + '&view=tgrid&id=' + number[1]
    htmltext = readUrl(url)
    course_detail = pd.read_html(htmltext)[1]
    data = pd.read_html(htmltext)[2]
    student = pd.concat([data[3],data[4],data[5]], axis = 1)
    course_detail = convertFirstRowToHeader(course_detail)
    student = convertFirstRowToHeader(student)
    temp = pd.concat([student, course_detail,pd.DataFrame({'termcode':[number[0]]})],axis = 1)
    temp.iloc[1,13] = temp.iloc[0,13]
    temp = temp[1:]
    x = student.index.size
    for i in range(0,x):
        temp.iloc[i,3:] =  temp.iloc[0,3:]
    df = pd.concat([df, temp])
    line = f.readline()

In [5]:
f.close()

In [6]:
df.to_csv('shibberu_course_detail.csv')